In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk.classify.util as util
from nltk.classify import NaiveBayesClassifier
from nltk.metrics import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder as BCF
import itertools
import pickle

import os.path
from statistics import mode
from nltk.classify import ClassifierI

nltk.download('punkt')


[nltk_data] Downloading package punkt to /Users/vincent/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
def featureEngineering(words):
    words = word_tokenize(words)
    scoreF = BigramAssocMeasures.chi_sq
    #bigram count
    n = 150
    bigrams = BCF.from_words(words).nbest(scoreF, n)
    #print(bigrams)
    return dict([word,True] for word in itertools.chain(words, bigrams))

In [13]:
pos_sen = open("positive.txt", 'r', encoding = 'latin-1').read()
neg_sen = open("negative.txt", 'r', encoding = 'latin-1').read()

emoji = open("emoji.txt",'r', encoding = 'latin-1').read()
pos_emoji = []
neg_emoji = []
for i in emoji.split('\n'):
    exp = ''
    if i[len(i)-2] == '-':
        for j in range(len(i) - 2):
            exp += i[j]
        neg_emoji.append(( {exp : True}, 'negative'))
    else:
        for j in range(len(i)-1):
            exp += i[j]
        pos_emoji.append(( {exp : True}, 'positive'))

prev = [(featureEngineering(words), 'positive') for words in pos_sen.split('\n')]
nrev = [(featureEngineering(words), 'negative') for words in neg_sen.split('\n')]

ncutoff = int(len(nrev)*3/4)
pcutoff = int(len(prev)*3/4)
train_set = nrev[:ncutoff] + prev[:pcutoff] + pos_emoji + neg_emoji
test_set = nrev[ncutoff:] + prev[pcutoff:]

pos_set = prev + pos_emoji
neg_set = nrev + neg_emoji

real_classifier = NaiveBayesClassifier.train(train_set)

# Saving classifier
save_doc = open("classifier.pickle", 'wb')
pickle.dump(real_classifier, save_doc)
save_doc.close()


In [14]:
#test_classifier = NaiveBayesClassifier.train(train_set)
fl = open('classifier.pickle','rb')
classifier = pickle.load(fl)
fl.close()
print ("Accuracy is : ", util.accuracy(classifier, test_set) * 100)

Accuracy is :  78.20705176294074
